<a href="https://colab.research.google.com/github/theindianwriter/CS6910-assignment_2/blob/main/DL_assignment_2_Part_A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np #for linear algebra operations
import os
import shutil
import matplotlib.pyplot as plt
from PIL import Image #for preprocessing the images

In [2]:
# PyTorch libraries
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

In [3]:
!pip install wandb -qqq
import wandb
wandb.login()

     |████████████████████████████████| 2.1MB 19.9MB/s 
     |████████████████████████████████| 163kB 57.3MB/s 
     |████████████████████████████████| 102kB 14.1MB/s 
     |████████████████████████████████| 133kB 57.2MB/s 
     |████████████████████████████████| 71kB 10.9MB/s 


<IPython.core.display.Javascript object>

wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter: ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
#enabling gpu 
device = "cpu"
if (torch.cuda.is_available()):
    device = "cuda"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!unzip -uq "/content/drive/My Drive/nature_12K.zip" -d "/content/drive/My Drive/"

In [5]:
training_folder_path = "/content/drive/MyDrive/inaturalist_12K/train"

In [6]:
#list down all the classes present in the dataset
classes = sorted([folder_name  for folder_name in os.listdir(training_folder_path) if not folder_name.startswith('.')])
print(classes)

['Amphibia', 'Animalia', 'Arachnida', 'Aves', 'Fungi', 'Insecta', 'Mammalia', 'Mollusca', 'Plantae', 'Reptilia']


In [7]:
# function to resize image
def resize_image(src_image, size=(128,128), bg_color="white"): 
    #creating a thumbnail of the image of the given size preserving the aspect ratio
    src_image.thumbnail(size,Image.ANTIALIAS)
    #creating a background image 
    new_image = Image.new("RGB", size, bg_color)
    #pasting the src image into it 
    new_image.paste(src_image, (int((size[0] - src_image.size[0]) / 2), int((size[1] - src_image.size[1]) / 2)))
    return new_image

In [8]:
new_training_folder_path = "./Natural_Dataset256/train"

In [10]:
image_size = (256,256)

if os.path.exists(new_training_folder_path):
    shutil.rmtree(new_training_folder_path)

for root,folders,_ in os.walk(training_folder_path):
    for folder in folders:
        print("resizing the images and saving for the folder ",folder)
        new_folder = os.path.join(new_training_folder_path,folder)

        if not os.path.exists(new_folder):
            os.makedirs(new_folder)
        
        file_names = os.listdir(os.path.join(root,folder))
        
        for file_name in file_names:
            if file_name.startswith('.'):
                continue
            file_path = os.path.join(root,folder,file_name)
            image = Image.open(file_path)
            resized_image = resize_image(image,image_size,"black")
            save_as = os.path.join(new_folder,file_name)
            resized_image.save(save_as)

print("resizing and saving done")

        


resizing the images and saving for the folder  Reptilia
resizing the images and saving for the folder  Mammalia
resizing the images and saving for the folder  Arachnida
resizing the images and saving for the folder  Plantae
resizing the images and saving for the folder  Aves
resizing the images and saving for the folder  Amphibia
resizing the images and saving for the folder  Insecta
resizing the images and saving for the folder  Animalia
resizing the images and saving for the folder  Mollusca
resizing the images and saving for the folder  Fungi
resizing and saving done


In [11]:
mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]

def load_dataset(dataset_path,data_augmentation = False,batch_size = 50):
    if data_augmentation:
        transformation = transforms.Compose([
            transforms.RandomResizedCrop(256),
            transforms.RandomHorizontalFlip(0.5),
            transforms.RandomVerticalFlip(0.3),
            transforms.ToTensor(),
        # Normalize the pixel values (in R, G, and B channels)
            transforms.Normalize(mean=mean, std=std)
        ])
    else:
        transformation = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize(mean=mean,std = std)                               
        ])

    full_dataset = torchvision.datasets.ImageFolder(
        root=dataset_path,
        transform=transformation
    )

    train_size = int(0.9 * len(full_dataset))
    test_size = len(full_dataset) - train_size

    train_dataset, test_dataset = torch.utils.data.random_split(full_dataset, [train_size, test_size])

    train_loader = torch.utils.data.DataLoader(
        train_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )

    test_loader = torch.utils.data.DataLoader(
        test_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False
    )
    return train_loader,test_loader

In [12]:
print(device)

cuda


In [13]:
class Network(nn.Module):

    def __init__(self,num_of_classes,filters,filter_size,activation_fn,stride_len = 1,batch_norm = True,padding_len = 1,max_pool_kernel_size = 2,max_pool_stride_len = 2,dropdout_ratio = 0.0,dense_neuron_size = 20000,img_size = (256,256)):
        super(Network,self).__init__()

        self.activation_fn = activation_fn
        self.batch_norm = batch_norm

        Out_dim_H = img_size[0]
        Out_dim_W = img_size[1]

        for F in filter_size:
            Out_dim_H =  int(((Out_dim_H - F + 2*padding_len)/stride_len)) + 1
            Out_dim_W =  int(((Out_dim_W - F + 2*padding_len)/stride_len)) + 1
            Out_dim_H =  int(((Out_dim_H - max_pool_kernel_size)/max_pool_stride_len))+ 1
            Out_dim_W =  int(((Out_dim_W - max_pool_kernel_size)/max_pool_stride_len)) + 1


        self.conv1 = nn.Conv2d(in_channels = 3,out_channels = filters[0],kernel_size = filter_size[0],stride = stride_len,padding = padding_len)
        self.conv2 = nn.Conv2d(in_channels = filters[0],out_channels = filters[1],kernel_size = filter_size[1],stride = stride_len,padding = padding_len)
        self.conv3 = nn.Conv2d(in_channels = filters[1],out_channels = filters[2],kernel_size = filter_size[2],stride = stride_len,padding = padding_len)
        self.conv4 = nn.Conv2d(in_channels = filters[2],out_channels = filters[3],kernel_size = filter_size[3],stride = stride_len,padding = padding_len)
        self.conv5 = nn.Conv2d(in_channels = filters[3],out_channels = filters[4],kernel_size = filter_size[4],stride = stride_len,padding = padding_len)

        self.pool = nn.MaxPool2d(kernel_size=max_pool_kernel_size,stride = max_pool_stride_len)

        self.drop = nn.Dropout2d(p=dropdout_ratio)
        
        if self.batch_norm:
            self.batch_norm1 = nn.BatchNorm2d(num_features=filters[0], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm2 = nn.BatchNorm2d(num_features=filters[1], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm3 = nn.BatchNorm2d(num_features=filters[2], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm4 = nn.BatchNorm2d(num_features=filters[3], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            self.batch_norm5 = nn.BatchNorm2d(num_features=filters[4], eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        

        self.post_conv_output_len = Out_dim_H*Out_dim_W*filters[4]
        #fully connected layer
        self.fc1 = nn.Linear(in_features = Out_dim_H*Out_dim_W*filters[4],out_features = dense_neuron_size)
        self.fc2 = nn.Linear(in_features = dense_neuron_size,out_features = num_of_classes)



    def perform_activation(self,fn,x):
        if fn == 'relu':
            return F.relu(x)
        elif fn == 'elu':
            return F.elu(x)
        elif fn == 'leaky_relu':
            return F.leaky_relu(x)
        elif fn == 'tanh':
            return F.tanh(x)
        elif fn == 'sigmoid':
            return F.sigmoid(x)

    def forward(self,x):
        if self.batch_norm:
            x = self.batch_norm1(self.conv1(x))
        else:
            x = self.conv1(x)

        x = self.pool(self.perform_activation(self.activation_fn[0],x))

        if self.batch_norm:
            x = self.batch_norm2(self.conv2(x))
        else:
            x = self.conv2(x)
    
        x = self.pool(self.perform_activation(self.activation_fn[1],x))

        if self.batch_norm:
            x = self.batch_norm3(self.conv3(x))
        else:
            x = self.conv3(x)


        x = self.pool(self.perform_activation(self.activation_fn[2],x))

        if self.batch_norm:
            x = self.batch_norm4(self.conv4(x))
        else:
            x = self.conv4(x)

        x = self.pool(self.perform_activation(self.activation_fn[3],x))
    
        if self.batch_norm:
            x = self.batch_norm5(self.conv5(x))
        else:
            x = self.conv5(x)

        x = self.pool(self.perform_activation(self.activation_fn[4],x))

        x = F.dropout(self.drop(x), training=self.training)

        x = x.view(-1, self.post_conv_output_len)

        x = F.relu(self.fc1(x))

        x = F.dropout(self.drop(x), training=self.training)

        x = self.fc2(x)
            
        return torch.log_softmax(x, dim=1)

In [14]:
def train(model,train_loader,optimizer,epoch,device):

    model.train()
    training_loss = 0
    batch_id = 0
    print("----------------------IN EPOCH {}--------------------------------".format(epoch))
    for data,target in train_loader:
        batch_id += 1
        data,target = data.to(device),target.to(device)

        optimizer.zero_grad()
        output = model(data)

        loss = loss_criteria(output,target)
        training_loss += loss.item()

        loss.backward()
        optimizer.step()

    print("avg training loss is {:.6f}".format(training_loss/batch_id))
    return training_loss


In [15]:
def test(model,test_loader,device):
    model.eval()
    test_loss = 0
    correct = 0
    batch_id = 0
    with torch.no_grad():
        for data,target in test_loader:
            batch_id += 1
            data,target = data.to(device),target.to(device)
            output = model(data)
            loss = loss_criteria(output,target)
            test_loss += loss.item()
            _,predicted = torch.max(output.data,1)
            correct += torch.sum(target==predicted).item()
    accuracy = 100*correct/ len(test_loader.dataset)
    print("avg val loss is {:.6f} and accuracy is {:.2f}%".format(test_loss/batch_id,accuracy))
    return test_loss,accuracy

    


In [29]:
model = Network(num_of_classes=10,filters = [32,64,128,128,128],filter_size = [3,3,3,3,3],activation_fn=['relu','relu','relu','relu','relu'],dropdout_ratio = 0.5,dense_neuron_size = 128,batch_norm = False).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.0002)

loss_criteria = nn.CrossEntropyLoss()
train_loader,val_loader = load_dataset(new_training_folder_path,True,100)
epochs = 10
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        test_loss = test(model, val_loader,device)
        torch.cuda.empty_cache()
        

Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.300061
avg val loss is 2.274428 and accuracy is 16.20%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.278487
avg val loss is 2.231518 and accuracy is 18.80%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.228630
avg val loss is 2.163601 and accuracy is 20.10%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.203314
avg val loss is 2.131405 and accuracy is 22.00%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.194650
avg val loss is 2.111873 and accuracy is 22.90%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.176996
avg val loss is 2.114474 and accuracy is 23.20%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.162044
avg val loss is 2.093028 and a

In [19]:
  torch.cuda.memory_summary(device=None, abbreviated=False)

'|===========================================================================|\n|                  PyTorch CUDA memory summary, device ID 0                 |\n|---------------------------------------------------------------------------|\n|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |\n|===========================================================================|\n|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |\n|---------------------------------------------------------------------------|\n| Allocated memory      |   24746 KB |    3295 MB |   15232 GB |   15232 GB |\n|       from large pool |   23168 KB |    3294 MB |   15062 GB |   15062 GB |\n|       from small pool |    1578 KB |      10 MB |     170 GB |     170 GB |\n|---------------------------------------------------------------------------|\n| Active memory         |   24746 KB |    3295 MB |   15232 GB |   15232 GB |\n|       from large pool |   23168 KB |    3294 MB |

In [17]:
defaults = dict(
  num_of_filter = 16,
  filter_size = 3,
  filter_organization = 2,
  activation_fn='relu',
  dropout_ratio = 0.3,
  dense_neuron_size = 64,
  lr = 0.0001,
  batch_size = 100,
  epochs = 10,
  data_augmentation = True,
  batch_norm = True
)

wandb.init(project="cs6910-assignment2-part_a",config = defaults)
config = wandb.config

wandb: Currently logged in as: theindianwriter (use `wandb login --relogin` to force relogin)


In [18]:
filters = []
filter_size = []
activation_fn = []  
rate = 1
for i in range(5):
    activation_fn.append(config.activation_fn)
    filter_size.append(config.filter_size)
    filters.append(config.num_of_filter*rate)
    rate *= config.filter_organization

model = Network(num_of_classes=10,filters = filters,filter_size = filter_size,activation_fn=activation_fn,dropdout_ratio = config.dropout_ratio,dense_neuron_size = config.dense_neuron_size,batch_norm = config.batch_norm).to(device)
optimizer = optim.Adam(model.parameters(), lr=config.lr)

loss_criteria = nn.CrossEntropyLoss()
train_loader,val_loader = load_dataset(new_training_folder_path,config.data_augmentation,config.batch_size)
epochs = config.epochs
print('Training on', device)
for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        val_loss,accuracy = test(model, val_loader,device)
        wandb.log({"epoch": epoch,"accuracy" : accuracy,"training loss":train_loss,"validation loss": val_loss })

Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.303739
avg val loss is 2.257631 and accuracy is 15.40%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.269994
avg val loss is 2.210450 and accuracy is 21.10%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.262568
avg val loss is 2.247485 and accuracy is 16.20%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.247239
avg val loss is 2.189405 and accuracy is 18.40%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.252020
avg val loss is 2.192552 and accuracy is 17.40%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.249447
avg val loss is 2.170791 and accuracy is 17.50%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.231398
avg val loss is 2.171901 and a

In [ ]:
sweep_config = {
    'method': 'random', #grid, random
    'metric': {
      'name': 'accuracy',
      'goal': 'maximize'   
    },
    'parameters': {
        'lr': {
            'values': [0.0001, 0.0002,0.0004]
        },
        'activation_fn': {
            'values': ['relu', 'tanh','leaky_relu']
        },
        'num_of_filters': {
            'values' : [32,64,256]
        },
        'filter_size' : {
            'values' : [3,5,7]
        },
        'filter_organization':{
            'values': [2,1,0.5]
        },
        'batch_size':{
            'values': [100,200]
        },
        'dropdout_ratio': {
            'values': [0.2,0.3,0.4,0.5]
        },
        'dense_neuron_size':{
            'values' : [256,128,64]
        },
        'data_augmentation':{
            'values': [True,False]
        },
        'epochs':{
            'values' : [10,5,15]
        },
        'batch_norm':{
            'values' : [True,False]
        },


    }
}

In [26]:
def experiment():
    defaults = dict(
        num_of_filter = 16,
        filter_size = 5,
        filter_organization = 2,
        activation_fn='leaky_relu',
        dropout_ratio = 0.3,
        dense_neuron_size = 64,
        lr = 0.0001,
        batch_size = 8,
        epochs = 10,
        data_augmentation = True,
        batch_norm = True
    )

    wandb.init(project="cs6910-assignment2-part_a",config = defaults)
    config = wandb.config

    filters = []
    filter_size = []
    activation_fn = []  
    if config.num_of_filters >= 256 and config.filter_organization == 2:
        config.filter_organization = 0.5
    rate = 1
    for i in range(5):
        activation_fn.append(config.activation_fn)
        filter_size.append(config.filter_size)
        filters.append(config.num_of_filter*rate)
        rate *= config.filter_organization

    model = Network(num_of_classes=10,filters = filters,filter_size = filter_size,activation_fn=activation_fn,dropdout_ratio = config.dropout_ratio,dense_neuron_size = config.dense_neuron_size,batch_norm = config.batch_norm).to(device)
    optimizer = optim.Adam(model.parameters(), lr=config.lr)

    loss_criteria = nn.CrossEntropyLoss()
    train_loader,val_loader = load_dataset(new_training_folder_path,config.data_augmentation,config.batch_size)
    epochs = config.epochs
    print('Training on', device)
    for epoch in range(1, epochs + 1):
        train_loss = train(model, train_loader, optimizer,epoch,device)
        val_loss,accuracy = test(model, val_loader,device)
        wandb.log({"epoch": epoch,"accuracy" : accuracy,"training loss":train_loss,"validation loss": val_loss })
        torch.cuda.empty_cache()

In [27]:
experiment()

epoch,10
accuracy,17.2
training loss,162.22196
validation loss,18.10518
_runtime,452
_timestamp,1618303906
_step,9


epoch,▁▂▃▃▄▅▆▆▇█
accuracy,▂▁▇▄▄▇▆█▅▇
training loss,█▄▄▄▂▃▂▂▁▂
validation loss,██▅▆▅▁▆▃▅▄
_runtime,▁▂▃▃▄▅▆▆▇█
_timestamp,▁▂▃▃▄▅▆▆▇█
_step,▁▂▃▃▄▅▆▆▇█


Training on cuda
----------------------IN EPOCH 1--------------------------------
avg training loss is 2.308392
avg val loss is 2.292873 and accuracy is 12.20%
----------------------IN EPOCH 2--------------------------------
avg training loss is 2.292382
avg val loss is 2.272477 and accuracy is 13.10%
----------------------IN EPOCH 3--------------------------------
avg training loss is 2.287171
avg val loss is 2.277719 and accuracy is 14.30%
----------------------IN EPOCH 4--------------------------------
avg training loss is 2.278473
avg val loss is 2.260938 and accuracy is 14.80%
----------------------IN EPOCH 5--------------------------------
avg training loss is 2.280558
avg val loss is 2.274638 and accuracy is 12.60%
----------------------IN EPOCH 6--------------------------------
avg training loss is 2.273916
avg val loss is 2.257609 and accuracy is 16.60%
----------------------IN EPOCH 7--------------------------------
avg training loss is 2.272362
avg val loss is 2.257060 and a

In [ ]:
sweep_id = wandb.sweep(sweep_config, entity="theindianwriter", project="cs6910-assignment2-part_a")

In [ ]:
wandb.agent(sweep_id, experiment)